## Categorizing US counties

In [1]:
import pandas as pd

### Import county category data
Source: NCHS Urban-Rural Classification Scheme for Counties

##### The NCHS has developed a six-level urban-rural classification scheme for U.S. counties and county-equivalent entities.
1. Metropolitan counties: Large central metro counties in MSA of 1 million population that: 1) contain the entire population of the largest principal city of the MSA, or 2) are completely contained within the largest principal city of the MSA, or 3) contain at least 250,000 residents of any principal city in the MSA.
2. Large fringe metro counties in MSA of 1 million or more population that do not qualify as large central 
3. Medium metro counties in MSA of 250,000-999,999 population.
4. Small metro counties are counties in MSAs of less than 250,000 population.
5. Micropolitan counties in micropolitan statistical area
6. Noncore counties not in micropolitan statistical areas
    
##### We can farther collapse these six categories into urban, suburban, and rural:
- Urban: 1 (Large central metro)
- Suburban: 2, 3, 4 (Large fringe metro counties, medium metro, and small metro counties)
- Rural: 5 and 6 (Micropolitan counties and Noncore counties)
    

In [35]:
county_categories = pd.read_excel("NCHSURCodes2013.xlsx")
print(county_categories.shape)
county_categories.head()

(3149, 9)


,FIPS code,State Abr.,County name,CBSA title,CBSA 2012 pop,County 2012 pop,2013 code,2006 code,1990-based code
0,1001,AL,Autauga County,"Montgomery, AL",377149,55514,3,3,3
1,1003,AL,Baldwin County,"Daphne-Fairhope-Foley, AL",190790,190790,4,5,3
2,1005,AL,Barbour County,NaN,.,27201,6,5,5
3,1007,AL,Bibb County,"Birmingham-Hoover, AL",1136650,22597,2,2,6
4,1009,AL,Blount County,"Birmingham-Hoover, AL",1136650,57826,2,2,3


In [36]:
selected = county_categories[["FIPS code", "2013 code"]]
selected.astype({"FIPS code": "int64"})
selected.head()

,FIPS code,2013 code
0,1001,3
1,1003,4
2,1005,6
3,1007,2
4,1009,2


### Import County-level Covid Data
Source: NY Times (https://github.com/nytimes/covid-19-data)

In [37]:
url = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"

covid_by_counties = pd.read_csv(url, error_bad_lines=False)

print(covid_by_counties.shape)
covid_by_counties.head()

(1131409, 6)


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


### Merging and Fixing

The first merge attempt shows that three of the counties in this dataset do not come with fips county code that identifies them. They are New York City, Kansas City, and Joplin. 

We will fill in the fips code for New York City with that of New York County because they generally refer to the same region. 

Joplin (Missouri) is a city in both Jasper and Newton County. Since both are designated by the NCHS system as small metro counties and Joplin is mostly in Jasper County, we will fill in the Joplin FIPS code with the Jasper County FIPS code.

Kansas City (Missouri) straddles the Kansas-Missouri state line. The bulk of it is in Jackson County, Missouri, but parts of it lie in Clay, Cass, and Platte Counties. We will fill in the Kansas City FIPS with that of Jackson County.

In [38]:
with_categories_left_1 = pd.merge(covid_by_counties, selected, left_on = "fips", right_on = "FIPS code", how = "left")
print(with_categories_left_1.shape)
with_categories_left_1[with_categories_left_1["fips"].isnull()].groupby("county").sum()

(1131409, 8)


,fips,cases,deaths,FIPS code,2013 code
county,,,,,
Joplin,0.0,679151,12893.0,0.0,0.0
Kansas City,0.0,5306163,66254.0,0.0,0.0
New York City,0.0,113877794,8081487.0,0.0,0.0
Unknown,0.0,12318961,516805.0,0.0,0.0


In [39]:
covid_by_counties.loc[covid_by_counties["county"] == "New York City", "fips"] = 36061
covid_by_counties.loc[covid_by_counties["county"] == "Joplin", "fips"] = 29097
covid_by_counties.loc[covid_by_counties["county"] == "Kansas City", "fips"] = 29095

In [40]:
with_categories_left_2 = pd.merge(covid_by_counties, selected, left_on = "fips", right_on = "FIPS code", how = "left")
print(with_categories_left_2.shape)
with_categories_left_2[with_categories_left_2["fips"].isnull()].groupby("county").sum()

(1131409, 8)


,date,county,state,fips,cases,deaths,FIPS code,2013 code
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0,53061.0,2.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0,53061.0,2.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0,53061.0,2.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0,17031.0,1.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0,53061.0,2.0


In [41]:
print(with_categories_left_2[with_categories_left_2["fips"].isnull()].shape[0])
print(with_categories_left_2[with_categories_left_2["fips"].isnull()].shape[0]/with_categories_left_2.shape[0])
print(with_categories_left_2[with_categories_left_2["fips"].isnull()].groupby("state").sum().shape)
print(with_categories_left_2[with_categories_left_2["fips"].isnull() == False].groupby("state").sum().shape)

9359
0.008271986522999198
(53, 5)
(54, 5)


After fixing the fips codes above, we are only missing county information on 9257 entries in the 1118423-entry dataset. That is only 0.83% of the data. We also check to make sure all 50 states are represented in our dataset.

### Dividing into Urban, Suburban, and Rural

In [53]:
with_categories_left_2.loc[with_categories_left_2["2013 code"] == 1, "category"] = "Urban"
with_categories_left_2.loc[with_categories_left_2["2013 code"] == 2, "category"] = "Suburban"
with_categories_left_2.loc[with_categories_left_2["2013 code"] == 3, "category"] = "Suburban"
with_categories_left_2.loc[with_categories_left_2["2013 code"] == 4, "category"] = "Suburban"
with_categories_left_2.loc[with_categories_left_2["2013 code"] == 5, "category"] = "Rural"
with_categories_left_2.loc[with_categories_left_2["2013 code"] == 6, "category"] = "Rural"

In [54]:
with_categories_left_2.head()

,date,county,state,fips,cases,deaths,FIPS code,2013 code,category
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0,53061.0,2.0,Urban
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0,53061.0,2.0,Urban
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0,53061.0,2.0,Urban
3,2020-01-24,Cook,Illinois,17031.0,1,0.0,17031.0,1.0,Urban
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0,53061.0,2.0,Urban


### Import College Data

In [16]:
college_url = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/colleges/colleges.csv"

covid_by_college = pd.read_csv(college_url, error_bad_lines=False)

print(covid_by_college.shape)
covid_by_college.head()

(1949, 9)


,date,state,county,city,ipeds_id,college,cases,cases_2021,notes
0,2021-02-26,Alabama,Madison,Huntsville,100654,Alabama A&M University,41,NaN,NaN
1,2021-02-26,Alabama,Montgomery,Montgomery,100724,Alabama State University,2,NaN,NaN
2,2021-02-26,Alabama,Limestone,Athens,100812,Athens State University,45,10.0,NaN
3,2021-02-26,Alabama,Lee,Auburn,100858,Auburn University,2499,324.0,NaN
4,2021-02-26,Alabama,Montgomery,Montgomery,100830,Auburn University at Montgomery,214,74.0,NaN


### Import total county cases data

In [18]:
county_total_url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/live/us-counties.csv'
covid_by_county_total = pd.read_csv(county_total_url, error_bad_lines=False)
print(covid_by_county_total.shape)
covid_by_county_total.head()

(3248, 10)


,date,county,state,fips,cases,deaths,confirmed_cases,confirmed_deaths,probable_cases,probable_deaths
0,2021-03-18,Autauga,Alabama,1001.0,6495,98.0,5563.0,87.0,932.0,11.0
1,2021-03-18,Baldwin,Alabama,1003.0,20287,296.0,14364.0,221.0,5923.0,75.0
2,2021-03-18,Barbour,Alabama,1005.0,2202,54.0,1227.0,37.0,975.0,17.0
3,2021-03-18,Bibb,Alabama,1007.0,2519,58.0,2035.0,35.0,484.0,23.0
4,2021-03-18,Blount,Alabama,1009.0,6376,130.0,4905.0,109.0,1471.0,21.0


### Merging and Fixing College Data with County Data

In [46]:
college_merge_with_county = covid_by_county_total.merge(covid_by_college, on = ['county','state'], how = 'outer')
college_merge_with_county = college_merge_with_county.merge(county_categories, left_on = 'fips', right_on = 'FIPS code')
college_merge_with_county = college_merge_with_county.drop(columns = ['CBSA title', '1990-based code', '2006 code', 'County name','State Abr.','FIPS code','cases_2021','notes','CBSA 2012 pop', 'date_y', 'city'])
college_merge_with_county = college_merge_with_county.rename({'cases_x': 'county_cases', 'cases_y': 'college_cases', 'date_x':'Date'}, axis=1)
print(college_merge_with_county.shape)

college_merge_with_county.head()


(4012, 15)


,Date,county,state,fips,county_cases,deaths,confirmed_cases,confirmed_deaths,probable_cases,probable_deaths,ipeds_id,college,college_cases,County 2012 pop,2013 code
0,2021-03-18,Autauga,Alabama,1001.0,6495.0,98.0,5563.0,87.0,932.0,11.0,NaN,NaN,NaN,55514,3
1,2021-03-18,Baldwin,Alabama,1003.0,20287.0,296.0,14364.0,221.0,5923.0,75.0,NaN,NaN,NaN,190790,4
2,2021-03-18,Barbour,Alabama,1005.0,2202.0,54.0,1227.0,37.0,975.0,17.0,NaN,NaN,NaN,27201,6
3,2021-03-18,Bibb,Alabama,1007.0,2519.0,58.0,2035.0,35.0,484.0,23.0,NaN,NaN,NaN,22597,2
4,2021-03-18,Blount,Alabama,1009.0,6376.0,130.0,4905.0,109.0,1471.0,21.0,NaN,NaN,NaN,57826,2
